In [ ]:
import cugraph
import cudf
import json
import operator

input_filename = "vk_network.csv"

print("Загрузка данных из CSV...")
df = cudf.read_csv(input_filename)
n_G = cugraph.from_cudf_edgelist(df, source='source', destination='target')

#(Betweenness Centrality)
print("Вычисление междуцентральности...")
betweenness = cugraph.betweenness_centrality(n_G)
betweenness_dict = dict(zip(betweenness['vertex'].to_pandas(), betweenness['betweenness_centrality'].to_pandas()))
sorted_betweenness = dict(sorted(betweenness_dict.items(), key=operator.itemgetter(1), reverse=True))
with open("top_nodes_betw.json", "w", encoding="utf-8") as f:
    json.dump(sorted_betweenness, f, ensure_ascii=False, indent=2)

#(Degree Centrality)
print("Вычисление степени центральности...")
degree = cugraph.degree_centrality(n_G)
degree_dict = dict(zip(degree['vertex'].to_pandas(), degree['degree_centrality'].to_pandas()))
sorted_degree = dict(sorted(degree_dict.items(), key=operator.itemgetter(1), reverse=True))
with open("top_nodes_degree.json", "w", encoding="utf-8") as f:
    json.dump(sorted_degree, f, ensure_ascii=False, indent=2)

# (Eigenvector Centrality)
print("Вычисление центральности по собственным векторам...")
eigenvector = cugraph.eigenvector_centrality(n_G, max_iter=100000, tol=1e-06)
eigenvector_dict = dict(zip(eigenvector['vertex'].to_pandas(), eigenvector['eigenvector_centrality'].to_pandas()))
sorted_eigenvector = dict(sorted(eigenvector_dict.items(), key=operator.itemgetter(1), reverse=True))
with open("top_nodes_eigen.json", "w", encoding="utf-8") as f:
    json.dump(sorted_eigenvector, f, ensure_ascii=False, indent=2)

print("\n ТОП узлы по центральностям сохранены в файлы:")
print("  - top_nodes_betw.json")
print("  - top_nodes_degree.json")
print("  - top_nodes_eigen.json")
